In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import paired_cosine_distances
from scipy.stats import pearsonr, spearmanr
import numpy as np
from sklearn.metrics import roc_auc_score
import pandas as pd

In [2]:
def cal_score(dataset, model):
    # dataset : [sentence1, sentence2, label(0 or 1)]
    sentence1, sentence2, label = zip(*dataset)
    
    embeddings1 = model.encode(sentence1, show_progress_bar=False, convert_to_numpy=True,normalize_embeddings=True)
    embeddings2 = model.encode(sentence2, show_progress_bar=False, convert_to_numpy=True,normalize_embeddings=True)
    labels = [int(x) for x in label]

    cosine_scores = 1 - (paired_cosine_distances(embeddings1, embeddings2))
    eval_auc_cosine = roc_auc_score(labels, cosine_scores)

    return eval_auc_cosine

In [3]:
model_name = 'kornwtp/ConGen-simcse-model-roberta-base-thai' # kornwtp/ConGen-simcse-model-roberta-base-thai
model = SentenceTransformer(model_name)
model.max_seq_length = 200

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/kornwtp_ConGen-simcse-model-roberta-base-thai. Creating a new one with MEAN pooling.


# Run data_set = 'test' and 'dev'

In [4]:
path = 'XNLI-1.0/xnli.{}.tsv' # download from https://dl.fbaipublicfiles.com/XNLI/XNLI-1.0.zip
data_set = 'test'

df = pd.read_csv(path.format(data_set),sep='\t')
df.loc[(df['language']=='th') & (df['gold_label']=='contradiction'),'label'] = '0'
df.loc[(df['language']=='th') & (df['gold_label']=='entailment'),'label'] = '1'

dataset = df[(df['language']=='th') & (df['gold_label']!='neutral')][['sentence1','sentence2','label']].values.tolist()

score = cal_score(dataset, model)
print(f"AUC score:{score:.4f}")

AUC score:0.6672


In [5]:
path = 'XNLI-1.0/xnli.{}.tsv' # download from https://dl.fbaipublicfiles.com/XNLI/XNLI-1.0.zip
data_set = 'dev'

df = pd.read_csv(path.format(data_set),sep='\t')
df.loc[(df['language']=='th') & (df['gold_label']=='contradiction'),'label'] = '0'
df.loc[(df['language']=='th') & (df['gold_label']=='entailment'),'label'] = '1'

dataset = df[(df['language']=='th') & (df['gold_label']!='neutral')][['sentence1','sentence2','label']].values.tolist()

score = cal_score(dataset, model)
print(f"AUC score:{score:.4f}")

AUC score:0.6719
